# Swindon Wellbeing

This script was written to provide easy interpretation of wellbeing statistics for Swindon. It has been initialised to load the data available [here](https://www.ons.gov.uk/datasets/wellbeing-local-authority/editions/time-series/versions/1) The link allows an excel file to be downloaded.

I make no claim that this script is overly efficient, it was made for personal interest.

Any questions please contact:
> Andrew Paul Barnes<br>
> Doctoral Student & Teaching Assistant<br>
> Department of Architecture and Civil Engineering<br>
> University of Bath<br>
> a.p.barnes@bath.ac.uk

## Libraries

To begin several libraries are imported to allow easy manipulation of the data.

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import date
from scipy.stats import linregress
from matplotlib.lines import Line2D

## Loading Data Required

First we load the data from the file indicated by the *MASTER_FILE* variable, the range of *MEASURES* is also defined to give the wellbeing measured to be extracted.

In [8]:
MASTER_FILE = "wellbeing_data.csv"
MEASURES = ["Anxiety", "Happiness", "Life Satisfaction", "Worthwhile"]

In [43]:
# Load data
wellbeing_data = pd.read_csv(MASTER_FILE, delimiter=',', skiprows=2)

def extract_average_data(data):
    """ Extracts the data from the PANDAS table into the measures of wellbeing. """
    measures = {}
    for m in MEASURES:
        # Get only this current measure
        m_data = data.loc[data['MeasureOfWellbeing'] == m]
        measures[m] = {}
        for score in m_data['Estimate'].unique():
            s_data = m_data.loc[m_data['Estimate'] == score]
            s_data = s_data.iloc[:, 4:].to_numpy()[0]
            measures[m][score] = np.zeros(int(s_data.shape[0]/4))
            for year in range(0, int(s_data.shape[0]/4)):
                measures[m][score][year] = s_data[year*4]
    return measures
wellbeing_data = extract_average_data(wellbeing_data)